In [21]:
import pandas as pd
import ollama
from enum import Enum
from typing import Optional
import random


df = pd.read_csv('breakthrough_technologies.csv')

systemPrompt = 'You are an expert in technology, economics, and social well-being, and are an editor for the MIT Technology Review. You are critical and discerning, with an eye for scientific accuracy. Make no reference to the prompt when providing your responses. Provide your responses directly without commentary.'

summarySystemPrompt = f'''{systemPrompt}
Every prompt will be an article about a technology written by the MIT Technology Review and why that technology will be a breakthrough. Provide a short summary of the article that describes the technology and why the MIT Technology Review picked it as a breakthrough technology in they year they did. Use this format:

"SUMMARY: [technology name] is a technology that does X, Y, and Z. The following industries are relevant to the tehnology: A, B, and C. 
IMPACT: [technology name] could improve human well-being by A, B, or C. It could impact the economy by A, B, or C. The expected timeline for this technology to have a large economic or social impact is 0-5 years / 5-15 years / 15-30 / 30+ years.
AUTHOR: [author name]
OPINION: [technology name] has had/not had the expected impact on social well-being or the economy since the year [year]. The impact has/has not been restricted to research and not widely available. For example, A, B, or C."
            
More info about the above sections:
SUMMARY should be 30 words and describe the technology, its potential uses, and relevant industries. All this should be derived directly from the article.
IMPACT should be 50 words, and about the expected impact of the technology and a specific timeline of expected impact, and any other relevant factors for why this technology is expected to be a breakthrough. This should all be derived from the article.
OPINION should be 50 words, and should be your opinion about whether the technology has had anything like the expected impact on social well-being or the economy since the year [year], provide a specific example or two to justify your claim. Be critical, differentiate between simply impact at a research level and impact on a wide-reaching social or economic level.

Do not provide any commentary or give any introduction to the summary, do not start with anything like "Here is a short summary of the article", just give the summary of the article in the above format. Be terse and direct with your sentences in order to fit more information into the summary. Ensure the end result is less than 150 words.
'''


class Model(Enum):
    phi3 = 'phi3:14b'
    gemma2 = 'gemma2:27b'
    # mistral = 'mistral-large:latest' #too slow
    llama31 = 'llama3.1:70b'
    qwen2 = 'qwen2:72b'
    deepseek = 'deepseek-llm:67b'

    def create(self):
        modelFile = f'''
            FROM {self.value}
            SYSTEM """{summarySystemPrompt}"""
            PARAMETER temperature 0.3
            PARAMETER num_ctx 8192
            PARAMETER seed 42
        '''
        print(modelFile)

        return ollama.create(model=f'MITTR_{self.value}', modelfile=modelFile, stream=False)

    def run(self, prompt: str) -> str:
        return ollama.generate(model=f'MITTR_{self.value}', prompt=prompt)["response"].strip()
    
class Phase(Enum):
    concept = 'concept'
    methodology = 'methodology'
    platform = 'platform'
    product = 'product'
    project = 'project'

    def description(self) -> str:
        match self:
            case Phase.concept: return 'When thinking about technologies, a concept is a nebulous idea that can have multiple different implementations. It is a thought or notion that is not yet concrete or fully developed, and multiple methodologies could be used to implement a concept. Technologies that could be described as concepts are things like: "AI for Everyone", and "The Sensing City".'
            case Phase.methodology: return 'A methodology is a system of methods used in a particular area of study or activity. It is a way of doing something, and it is a set of principles, tools, and practices that are used to solve problems. Technologies that could be described as methodologies are things like: "CRISPR", and "The Blockchain".'
            case Phase.platform: return 'A platform is a system that allows multiple technologies or products to be built on top of it. It is a base that other technologies can be built on, and it is a foundation that other technologies can use. Technologies that could be described as platforms are things like: "The Internet", and "The Cloud".'
            case Phase.product: return 'A product is a tangible item that is created through a process. It is something that can be bought, sold, or used, and it is something that has a specific function or purpose. Technologies that could be described as products are things like: "The iPhone", and "Slack".'
            case Phase.project: return 'A project is a standalone endeavor that is undertaken to create a unique product, service, or result. It is something that has a specific goal or objective, and it is something that has a defined beginning and end. Technologies that could be described as projects are things like: "The Human Genome Project", and "James Webb Space Telescope".'
        return 'Unknown phase'


class Technology:
    def __init__(self, name: str, year: str, link: str, specific_link: Optional[str], blurb: str):
        self.name = name
        self.year = year
        self.link = link
        self.specific_link = specific_link
        self.blurb = blurb

    def description(self, model: Model) -> str:
        return model.run(f'provide a short description, less than 50 words, of the technology "{self.name}", use this article as context: "{self.blurb}".')
    
    def author(self, model: Model) -> str:
        return model.run(f'extract the author of the following blurb. return only the name or "n/a" if unknown or no blurb is provided. only provide the full name, no other information. The author is probably at the end of the article. Here is the article: "{self.blurb}"')
    
    def reasoning(self, model: Model) -> str:
        return model.run(f'provide a short bullet list of the reasoning behind why MIT Technology Review picked "{self.name}" as a breakthrough technology for the year {self.year} using the content from this article: "{self.blurb}"')
    
    def predictions(self, model: Model) -> str:
        return model.run(f'provide a bullet list of the predictions the MIT Technology Review made about "{self.name}" in this article: "{self.blurb}"')
    
    def impact(self, model: Model) -> str:
        return model.run(f'provide a bullet list of the impact that "{self.name}" has had on society since the year {self.year}, using this article as context: "{self.blurb}"')
    
    def sectors(self, model: Model) -> str:
        return model.run(f'provide a comma-separated list of sectors that "{self.name}" has impacted since the year {self.year}. Do not give any commentary, just list the sectors. Sectors should also include government policy, healthcare, etc. Use this article as context: "{self.blurb}"')
    
    def phase(self, model: Model) -> str:
        prompt= f'''
        this is a list of forms with descriptions in the format "form: description": {list(map(lambda phase: f'{phase.value}: {phase.description()}', [phase for phase in Phase]))}
         
        pick one those "form" terms to best describe the technology "{self.name}".  
        
        Just return one of those words, do not provide commentary. 
        
        Use this article as context: "{self.blurb}"'''
        return model.run(prompt)
    
    def gdpImpact(self, model: Model) -> str:
        return model.run(f'give your best attempt at a guess of what % the technology "{self.name}" has impacted the US GDP since the year {self.year}. Only provide the % as a range within one order of magnitude, no commentary or extra information.')
    
    def outlook(self, model: Model) -> str:
        return model.run(f'Choose exactly one of "positive" or "negative" to describe the outlook for the technology {self.name}" as described in the article. "Positive" means that the expectation is that this technology will help the world, "negative" means that the expectation is that this technology may hurt or have be impactful in a non-positive way. Only respond with "positive" or "negative", no extra commentary. Here is the article: "{self.blurb}"')
    
    def type(self, model: Model) -> str:
        return model.run(f'pick one of "hardware", "software", "biotech", "nanotech", "climate tech", or "other" to best describe the technology "{self.name}". Just return one of those tags, do not provide commentary.')
    
    def expected_timeline(self, model: Model) -> str:
        return model.run(f'provide one of the following "0-10 years", "10-50 years", "50+ years" to best describe the expected timeline for "{self.name}" to be an impactful technology as described in this article. Just return one of the timelines, no commentary. Here is the article: "{self.blurb}"')
    
    def summarize(self, model: Model) -> str:
        summary = model.run(f'''Here is the article to summarize about {self.name} from the year {self.year}: "{self.blurb}"''')
        # summary = model.run(f'''
        #                 Provide a short summary of the following article that describes the technology "{self.name}" and why the MIT Technology Review picked it as a breakthrough technology in {self.year}. Use this format:
                        
        #                 "SUMMARY: {self.name} is a technology that does X, Y, and Z. The following industries are relevant to the tehnology: A, B, and C. 
        #                 IMPACT: {self.name} could improve human well-being by A, B, or C. It could impact the economy by A, B, or C. The expected timeline for this technology to have a large economic or social impact is A, B, or C.
        #                 AUTHOR: [author name]
        #                 OPINION: {self.name} has had/not had the expected impact on social well-being or the economy since the year {self.year}. The impact has/has not been restricted to research and not widely available. For example, A, B, or C."

        #                 SUMMARY should describe the technology, its potential uses, and relevant industries. All this should be derived directly from the article.
        #                 IMPACT should be about the expected impact of the technology and the timeline of expected impact, and any other relevant factors for why this technology is a breakthrough. This should all be derived from the article.
        #                 OPINION should be your opinion about whether the technology has had anything like the expected impact on social well-being or the economy since the year {self.year}, provide a specific example or two to justify your claim. Be critical, differentiate between simply impact at a research level and impact on a wide-reaching social or economic level.

        #                 Do not provide any commentary or give any introduction to the summary, do not start with anything like "Here is a short summary of the article", just give the summary of the article. Be terse and direct with your sentences in order to fit more information into the summary. Ensure the end result is less than 150 words.

        #                 Here is the article to summarize: "{self.blurb}"
        #                 ''')
        print(summary)
        return summary
    

import time

class ExpandedTechnology:
    def __init__(self, technology: Technology, model: Model):
        start_time = time.time()
        self.name = technology.name
        self.year = technology.year
        self.link = technology.link
        self.specific_link = technology.specific_link
        # self.blurb = technology.blurb
        # self.description = technology.description(model)
        # self.author = technology.author(model)
        # self.gdp_impact = technology.gdpImpact(model)
        # self.outlook = technology.outlook(model)
        # self.type = technology.type(model)
        # self.phase = technology.phase(model)
        # self.expected_timeline = technology.expected_timeline(model)
        extractions = technology.summarize(model)
        self.summary = ollama.generate(model=Model.llama31.value, prompt=f'Do not add any text of your own, and ignore the "IMPACT:", "AUTHOR:" and "OPINION:" sections. Just return the "SUMMARY:" text verbatim from following text: "{extractions}"')["response"].strip()
        self.impact = ollama.generate(model=Model.llama31.value, prompt=f'Do not add any text of your own, and ignore the "SUMMARY:", "AUTHOR:" and "OPINION:" sections. Just return the "IMPACT:" text verbatim from following text: "{extractions}"')["response"].strip()
        self.author = ollama.generate(model=Model.llama31.value, prompt=f'Do not add any text of your own, and ignore the "SUMMARY:", "IMPACT:" and "OPINION:" sections. Just return the "AUTHOR:" text verbatim from following text: "{extractions}"')["response"].strip()
        self.opinion = ollama.generate(model=Model.llama31.value, prompt=f'Do not add any text of your own, and ignore the "SUMMARY:", "IMPACT:" and "AUTHOR:" sections. Just return the "OPINION:" text verbatim from following text: "{extractions}"')["response"].strip()
        print(self.summary)
        print(self.impact)
        print(self.author)
        print(self.opinion)
        print('time taken for ', self.name, ': ', time.time() - start_time)




technologies: list[Technology] = df.apply(
    lambda row: Technology(name=row['name'], 
                           year=row['year'], 
                           link=row['link'], 
                           specific_link=row['specific_link'], 
                           blurb=row['blurb']), 
    axis=1
)
                      
# random.shuffle(technologies)
# for technology in technologies:
#     print(technology.name)
#     for model in Model:
#         print('****', model.value, '****')
#         print(ExpandedTechnology(technology, model).__dict__)
#         # print('description: \n', technology.description(model))
#         # print('author: \n', technology.author(model))
#         # print('reasoning: ', technology.reasoning(model))
#         # print('predictions: \n', technology.predictions(model))
#         # print('impact: \n', technology.impact(model))
#         # print('sectors: \n', technology.sectors(model))
#         # print('form: \n', technology.form(model))

#         # gdpImpact = technology.gdpImpact(model)
#         # print('gdpImpact: \n', gdpImpact)
#         # print('gdp %: \n', model.run(f'extract the final gdp impact % (only provide the % as a range within one order of magnitude, no commentary or extra information) from this explanation: "{gdpImpact}"'))

#     print("----------------------")




llama31 = Model.llama31.create()
print(llama31)


# map technologies to exapnded technologies
# technologies = technologies[:2]
expanded_techs = list(map(lambda technology: ExpandedTechnology(technology, Model.llama31), technologies))

# output expanded_techs as csv
expanded_df = pd.DataFrame([tech.__dict__ for tech in expanded_techs])
expanded_df.to_csv('expanded_breakthrough_technologies.csv', index=False)
print(expanded_df)



            FROM llama3.1:70b
            SYSTEM """You are an expert in technology, economics, and social well-being, and are an editor for the MIT Technology Review. You are critical and discerning, with an eye for scientific accuracy. Make no reference to the prompt when providing your responses. Provide your responses directly without commentary.
Every prompt will be an article about a technology written by the MIT Technology Review and why that technology will be a breakthrough. Provide a short summary of the article that describes the technology and why the MIT Technology Review picked it as a breakthrough technology in they year they did. Use this format:

"SUMMARY: [technology name] is a technology that does X, Y, and Z. The following industries are relevant to the tehnology: A, B, and C. 
IMPACT: [technology name] could improve human well-being by A, B, or C. It could impact the economy by A, B, or C. The expected timeline for this technology to have a large economic or socia

KeyboardInterrupt: 